1.What are the advantages of a CNN over a fully connected DNN for image classification?

CNN's layers are only partially connnected and reuses its weights.
learned a kernel which can detect a particular features.
A CNN's architecture embeds this prior knowledge.

2.Consider a CNN composed of three convolutional layers, each with 3 × 3 kernels, a stride of
2, and &quot;same&quot; padding. The lowest layer outputs 100 feature maps, the middle one outputs
200, and the top one outputs 400. The input images are RGB images of 200 × 300 pixels.
What is the total number of parameters in the CNN? If we are using 32-bit floats, at least how much
RAM will this network require when making a prediction for a single instance? What about when
training on a mini-batch of 50 images?

parameters

first convolutional layer kernel-size and RGB channels, plus bias: 3 * 3 * 3 + 1 = 28 output feature maps is 100: 28 * 100 = 2800
second convolutional layer kernel-size and last feature maps, plus bias: 3 * 3 * 100 + 1 = 901 output feature maps is 200: 901 * 200 = 180200
third convolutional layers kernel-size and last feautre maps, plus bias: 3 * 3 * 200 + 1 =1801 output feautre maps is 400: 1801 * 400 = 720400
Total parameters is 2800 + 180200 + 720400 = 903400

memories since 32-bit is 4 bytes

first convolutional layer one feature map size: 100 * 150 = 15000 total output: 15000 * 100 = 1,500,000
second convolutional layer one feature map size: 50 * 75 = 3,750 total output: 3750 * 200 = 750,000
third convolutional layer one feature map size: 25 * 38 = 950 total ouput: 950 * 400 = 380, 000
(1,500,000 + 750,000 + 380,000) * 4 / 1024 /1024 = 10.032 (MB) 903400 * 4 / 1024 / 1024 = 3.44 (MB) 10.032+ 3.44=13.47(MB)

3.If your GPU runs out of memory while training a CNN, what are five things you could try to
solve the problem?

reduce mini-batch size
reduce dimensionality using a larger stride in one or more layers
remove one or more layers
using 16-bits instead of 32-bit floats
distributed the cnn across multiple devices

4.Why would you want to add a max pooling layer rather than a convolutional layer with the
same stride?

A max pooling layer has no parameters at all, whereas a convolutional layer has quite a few.

5.When would you want to add a local response normalization layer?

Normalization layers in the model often helps to speed up and stabilize the learning process. If training with large batches isn't an issue and if the network doesn't have any recurrent connections, Batch Normalization could be used.

6.Can you name the main innovations in AlexNet, compared to LeNet-5? What about the main
innovations in GoogLeNet, ResNet, SENet, and Xception?

AlexNet:

it is much larger and deeper
stacks convolutional layer directly on top of each convlutional layer

GooLeNet:

introduce a inception modules, which make it possible to have much deeper net than previous network

ResNet:

introduce a skip connection.

7.What is a fully convolutional network?

FCNs, or Fully Convolutional Networks, are a form of architecture that is primarily used for semantic segmentation. Convolution, pooling, and upsampling are the only locally linked layers they use. Since dense layers aren't used, there are fewer parameters (making the networks faster to train).

How can you convert a dense layer into a
convolutional layer?

A fully convolution network can be built by simply replacing the FC layers with there equivalent Conv layers. One way to do so is to pop layers from the model. In the model stack, each popping will remove the last layer.

8.What is the main technical difficulty of semantic segmentation?

One of the most non-trivial tasks in image processing is segmentation. Segmentation is the process defining an image in such a manner that different objects can be extracted from it. In it's simplest form, segmentation exists as a thresholding problem.

9.Build your own CNN from scratch and try to achieve the highest possible accuracy on MNIST.

In [1]:
# First of all importing all required libraries

import numpy as np
import keras
from keras.datasets import mnist
from keras.models import Model
from keras.layers import Dense, Input
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten
from keras import backend as k

Creating the train data and test data
Test data: Used for testing the model that how our model has been trained. 
Train data: Used to train our model.

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [7]:
# Checking data format

img_rows, img_cols=28, 28
 
if k.image_data_format() == 'channels_first':
   x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
   x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
   inpx = (1, img_rows, img_cols)
 
else:
   x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
   x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
   inpx = (img_rows, img_cols, 1)
 
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

#description of the output classes

y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)

In [9]:
#Now, the dataset is ready so let’s move towards the CNN model 

inpx = Input(shape=inpx)
layer1 = Conv2D(32, kernel_size=(3, 3), activation='relu')(inpx)
layer2 = Conv2D(64, (3, 3), activation='relu')(layer1)
layer3 = MaxPooling2D(pool_size=(3, 3))(layer2)
layer4 = Dropout(0.5)(layer3)
layer5 = Flatten()(layer4)
layer6 = Dense(250, activation='sigmoid')(layer5)
layer7 = Dense(10, activation='softmax')(layer6)
 

#Calling compile and fit function
model = Model([inpx], layer7)
model.compile(optimizer=keras.optimizers.Adadelta(),
              loss=keras.losses.categorical_crossentropy,
              metrics=['accuracy'])
 
model.fit(x_train, y_train, epochs=12, batch_size=500)

#Evaluating function

score = model.evaluate(x_test, y_test, verbose=0)
print('loss=', score[0])
print('accuracy=', score[1])

loss = 0.0295960184669
accuracy = 0.991

10.Use transfer learning for large image classification, going through these steps:


a. Create a training set containing at least 100 images per class. For example, you could
classify your own pictures based on the location (beach, mountain, city, etc.), or
alternatively you can use an existing dataset (e.g., from TensorFlow Datasets).

import sys
import sklearn
import tensorflow as tp
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np
import os
np.random.seed(42)
tf.random.set_seed(42)

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

PROJECT_ROOT_DIR = "."
CHAPTER_ID = "cnn"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)


dataset, info = tfds.load("food101",
                                             shuffle_files=False,
                                             with_info=True,
                                             as_supervised=True)

Downloading and preparing dataset 4.65 GiB (download: 4.65 GiB, generated: Unknown size, total: 4.65 GiB) to /root/tensorflow_datasets/food101/2.0.0...




Dataset food101 downloaded and prepared to /root/tensorflow_datasets/food101/2.0.0. Subsequent calls will reuse this data.
2022-05-22 01:42:56.525982: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-22 01:42:56.648735: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-22 01:42:56.649537: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-22 01:42:56.653453: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-22 01:42:56.655431: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-22 01:42:56.656437: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-22 01:42:56.657378: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-22 01:42:58.789808: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-22 01:42:58.790612: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-22 01:42:58.791290: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-22 01:42:58.793145: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15403 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0

class_names = info.features["label"].names
class_names

['apple_pie',
 'baby_back_ribs',
 'baklava',
 'beef_carpaccio',
 'beef_tartare',
 'beet_salad',
 'beignets',
 'bibimbap',
 'bread_pudding',
 'breakfast_burrito',
 'bruschetta',
 'caesar_salad',
 'cannoli',
 'caprese_salad',
 'carrot_cake',
 'ceviche',
 'cheesecake',
 'cheese_plate',
 'chicken_curry',
 'chicken_quesadilla',
 'chicken_wings',
 'chocolate_cake',
 'chocolate_mousse',
 'churros',
 'clam_chowder',
 'club_sandwich',
 'crab_cakes',
 'creme_brulee',
 'croque_madame',
 'cup_cakes',
 'deviled_eggs',
 'donuts',
 'dumplings',
 'edamame',
  'eggs_benedict',
 'escargots',
 'falafel',
 'filet_mignon',
 'fish_and_chips',
 'foie_gras',
 'french_fries',
 'french_onion_soup',
 'french_toast',
 'fried_calamari',
 'fried_rice',
 'frozen_yogurt',
 'garlic_bread',
 'gnocchi',
  'greek_salad',
 'grilled_cheese_sandwich',
 'grilled_salmon',
 'guacamole',
 'gyoza',
 'hamburger',
 'hot_and_sour_soup',
 'hot_dog',
 'huevos_rancheros',
 'hummus',
 'ice_cream',
 'lasagna',
 'lobster_bisque',
 'lobster_roll_sandwich',
 'macaroni_and_cheese',
 'macarons',
 'miso_soup',
 'mussels',
 'nachos',
 'omelette',
 'onion_rings',
 'oysters',
 'pad_thai',
 'paella',
 'pancakes',
 'panna_cotta',
 'peking_duck',
 'pho',
 'pizza',
 'pork_chop',
 'poutine',
 'prime_rib',
 'pulled_pork_sandwich',
 'ramen',
 'ravioli',
 'red_velvet_cake',
 'risotto',
 'samosa',
 'sashimi',
 'scallops',
 'seaweed_salad',
 'shrimp_and_grits',
 'spaghetti_bolognese',
 'spaghetti_carbonara',
 'spring_rolls',
 'steak',
 'strawberry_shortcake',
 'sushi',
 'tacos',
 'takoyaki',
 'tiramisu',
 'tuna_tartare',
 'waffles']

n_classes = info.features["label"].num_classes
n_classes

101

dataset_size = info.splits["train"].num_examples
dataset_size

75750

In food101, train dataset contains 75750 images(750 images per class), while test dataset contains 2520 images(250 images per class) over 101 classes.

#b. Split it into a training set, a validation set, and a test set.
test_set_raw, valid_set_raw, train_set_raw = tfds.load(
    "food101",
    split=["train[:5%]", "train[5%:15%]", "train[15%:40%]"],
    as_supervised=True,
    shuffle_files=True)

import matplotlib.pyplot as plt 
%matplotlib inline
plt.figure(figsize=(12, 10))
index = 0
for image, label in train_set_raw.take(9):
    index += 1
    plt.subplot(3, 3, index)
    plt.imshow(image)
    plt.title("Class: {}".format(class_names[label]))
    plt.axis("off")

plt.show()

c. Build the input pipeline, including the appropriate preprocessing operations, and
optionally add data augmentation.

In [20]:
# crop the image and standardize the shape to 224*224*3
def central_crop(image):
    shape = tf.shape(image)
    min_dim = tf.reduce_min([shape[0], shape[1]])
    top_crop = (shape[0] - min_dim) // 4
    bottom_crop = shape[0] - top_crop
    left_crop = (shape[1] - min_dim) // 4
    right_crop = shape[1] - left_crop
    return image[top_crop:bottom_crop, left_crop:right_crop]

def random_crop(image):
    shape = tf.shape(image)
    min_dim = tf.reduce_min([shape[0], shape[1]]) * 90 // 100
    return tf.image.random_crop(image, [min_dim, min_dim, 3])
def preprocess(image, label, randomize=False):
    if randomize:
        cropped_image = random_crop(image)
        cropped_image = tf.image.random_flip_left_right(cropped_image)
    else:
        cropped_image = central_crop(image)
    resized_image = tf.image.resize(cropped_image, [224, 224])
    final_image = keras.applications.xception.preprocess_input(resized_image)
    return final_image, label

In [21]:
from functools import partial

batch_size = 32
train_set = train_set_raw.shuffle(1000).repeat()
train_set = train_set.map(partial(preprocess, randomize=True)).batch(batch_size).prefetch(1)
valid_set = valid_set_raw.map(preprocess).batch(batch_size).prefetch(1)
test_set = test_set_raw.map(preprocess).batch(batch_size).prefetch(1)

plt.figure(figsize=(12, 12))
for X_batch, y_batch in train_set.take(1):
    for index in range(9):
        plt.subplot(3, 3, index + 1)
        plt.imshow(X_batch[index] / 2 + 0.5)
        plt.title("Class: {}".format(class_names[y_batch[index]]))
        plt.axis("off")

plt.show()

#d. Fine-tune a pretrained model on this dataset.

base_model = keras.applications.xception.Xception(weights="imagenet",
                                                  include_top=False)
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(n_classes, activation="softmax")(avg)
model = keras.models.Model(inputs=base_model.input, outputs=output)

Downloading data from https://storage.googleapis.com/tensorflow/keras-applications/xception/xception_weights_tf_dim_ordering_tf_kernels_notop.h5
83689472/83683744 [==============================] - 2s 0us/step
83697664/83683744 [==============================] - 2s 0us/step

base_model = keras.applications.xception.Xception(weights="imagenet",
                                                  include_top=False)
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(n_classes, activation="softmax")(avg)
model = keras.models.Model(inputs=base_model.input, outputs=output)
for layer in base_model.layers:
    layer.trainable = False

optimizer = keras.optimizers.SGD(learning_rate=0.05, momentum=0.9, decay=0.01)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])

history = model.fit(train_set,
                    steps_per_epoch=int(0.75 * dataset_size / batch_size),
                    validation_data=valid_set,
                    validation_steps=int(0.15 * dataset_size / batch_size),
                    epochs=10)

base_model = keras.applications.xception.Xception(weights="imagenet",
                                                  include_top=False)
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(n_classes, activation="softmax")(avg)
model = keras.models.Model(inputs=base_model.input, outputs=output)
for layer in base_model.layers:
    layer.trainable = False

optimizer = keras.optimizers.SGD(learning_rate=0.05, momentum=0.9, decay=0.01)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
history = model.fit(train_set,
                    steps_per_epoch=int(0.75 * dataset_size / batch_size),
                    validation_data=valid_set,
                    validation_steps=int(0.15 * dataset_size / batch_size),
                    epochs=10)

Epoch 1/10
1775/1775 [==============================] - 181s 100ms/step - loss: 1.9680 - accuracy: 0.5095 - val_loss: 1.9272 - val_accuracy: 0.5145
Epoch 2/10
1775/1775 [==============================] - 154s 87ms/step - loss: 1.6239 - accuracy: 0.5899
Epoch 3/10
1775/1775 [==============================] - 152s 85ms/step - loss: 1.5669 - accuracy: 0.6034
Epoch 4/10
1775/1775 [==============================] - 151s 85ms/step - loss: 1.5321 - accuracy: 0.6136
Epoch 5/10
1775/1775 [==============================] - 151s 85ms/step - loss: 1.5069 - accuracy: 0.6203
Epoch 6/10
1775/1775 [==============================] - 152s 86ms/step - loss: 1.4944 - accuracy: 0.6243
Epoch 7/10
1775/1775 [==============================] - 153s 86ms/step - loss: 1.4795 - accuracy: 0.6263
Epoch 8/10
1775/1775 [==============================] - 153s 86ms/step - loss: 1.4738 - accuracy: 0.6269
Epoch 9/10
1775/1775 [==============================] - 154s 87ms/step - loss: 1.4633 - accuracy: 0.6297
Epoch 10/10
1775/1775 [==============================] - 155s 87ms/step - loss: 1.4541 - accuracy: 0.6318